<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
# download
!curl -O https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  249M  100  249M    0     0  1112k      0  0:03:49  0:03:49 --:--:--  474k01k      0  0:06:03  0:00:06  0:05:57  867k 0  0:03:30  0:00:53  0:02:37  521k 0  0:04:00  0:01:18  0:02:42  905k  0:03:58  0:01:39  0:02:19 1228k 0     0  1088k      0  0:03:54  0:01:46  0:02:08 1339k  0     0  1193k      0  0:03:33  0:03:11  0:00:22  904k 1127k      0  0:03:46  0:03:44  0:00:02  585k:45  0:00:02  555k


In [ ]:
# unzip
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [11]:
import keras

In [12]:
# Global parameters
num_classes = 2
batch_size = 100
img_target_size = 224

In [13]:
# Image Data generator
data_gen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.vgg16.preprocess_input
)

# Training data generator
train_gen = data_gen.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(img_target_size, img_target_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Test data generator
test_gen = data_gen.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(img_target_size, img_target_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [14]:
# Define the model
model_vgg = keras.models.Sequential()

# base model: VGG16
model_vgg.add(keras.applications.VGG16(include_top=False, pooling='avg'))

# output layer
model_vgg.add(keras.layers.Dense(num_classes, activation='softmax'))

# show layers
model_vgg.layers

In [15]:
# Set VGG16 non-trainable
model_vgg.layers[0].trainable=False

model_vgg.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [16]:
# Compile the model
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Fit the model
num_epochs = 2

# Use smaller set for training due to performance constraints on my local machine !!
steps_train = 30
steps_test = 10

fit_history_vgg = model_vgg.fit_generator(
    train_gen,
    steps_per_epoch=steps_train,
    epochs=num_epochs,
    validation_data=test_gen,
    validation_steps=steps_test,
    verbose=1,
)

Epoch 1/2
30/30 [==============================] - 1788s 60s/step - loss: 0.3820 - accuracy: 0.8210 - val_loss: 0.1496 - val_accuracy: 0.9660
Epoch 2/2
30/30 [==============================] - 1822s 61s/step - loss: 0.0937 - accuracy: 0.9860 - val_loss: 0.0647 - val_accuracy: 0.9890


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [41]:
# Load saved resnet model
model_resnet = keras.models.load_model('classifier_resnet_model.h5')

/Users/andreas/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [49]:
# Generate test data
data_gen = keras.preprocessing.image.ImageDataGenerator()

test_gen = data_gen.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(img_target_size, img_target_size),
    shuffle=False
)

Found 500 images belonging to 2 classes.


In [50]:
# Evaluate both models
model_resnet_eval = model_resnet.evaluate_generator(test_gen)
model_vgg_eval = model_vgg.evaluate_generator(test_gen)

In [53]:
# Resnet score
model_resnet_eval[1]

0.5519999861717224

In [54]:
# VGG score
model_vgg_eval[1]

0.7120000123977661

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [63]:
test_gen = data_gen.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(img_target_size, img_target_size),
    shuffle=True
)

Found 500 images belonging to 2 classes.


In [67]:
# Predict generator
model_vgg_predict = model_vgg.predict_generator(test_gen, steps=1)

In [68]:
# First five predictions VGG
["Negative" if model_vgg_predict[i][0]>model_resnet_predict[i][1] else "Positve" for i in range(5)]

['Positve', 'Negative', 'Negative', 'Positve', 'Negative']

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).